In [1]:
import requests
import datetime as dt
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, DateTime


In [2]:
engine = create_engine('sqlite:///db.sqlite')
Base = declarative_base()

class Hits(Base):
    __tablename__ = "hits"

    hit_id = Column(Integer, nullable=False, unique=True, autoincrement=True, primary_key=True)
    year = Column(Integer)
    month = Column(String)
    day = Column(Integer)
    day_of_week = Column(String)
    nhits = Column(Integer)


Base.metadata.create_all(engine)

In [3]:
from sqlalchemy.orm import sessionmaker


Session = sessionmaker(bind=engine)
session = Session()


import sqlite3
database = sqlite3.connect("db.sqlite")
cur = database.cursor()

years = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

for year in years :
    for month in months:
        URL = f"https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=&rows=10000&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&refine.gc_obo_gare_origine_r_name=Lille+Europe&refine.date={year}%2F{month}"
        d = requests.get(URL).json()

        if 'facet_groups' in d :
            hits = d["facet_groups"][0]['facets'][0]['facets'][0]['facets']

            for hit in hits:
                    year = int(year)
                    month = int(month)
                    day = int(hit['name'])
                    day_of_week = (dt.date(year, month, day)).strftime("%A")
                    nhits = int(hit['count'])
                    session.add(Hits(year=year, month=month, day=day,day_of_week=day_of_week, nhits=nhits))

session.commit()